In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# How many reads are in the raw files?
df1 = pd.read_csv('../data/preprocessing/QC-Demux/multiqc_data/multiqc_cutadapt.txt', sep='\t')
df2 = pd.read_csv('../data/preprocessing/QC-Demux/multiqc_data_1/multiqc_cutadapt.txt', sep='\t')
df3 = pd.read_csv('../data/preprocessing/QC-Demux/multiqc_data_2/multiqc_cutadapt.txt', sep='\t')
df_raw = pd.concat([df1, df2, df3]).reset_index()

# How many reads per sample after demux?
df1 = pd.read_csv('../data/preprocessing/QC-Demux/multiqc_data/multiqc_fastqc.txt', sep='\t')
df2 = pd.read_csv('../data/preprocessing/QC-Demux/multiqc_data_1/multiqc_fastqc.txt', sep='\t')
df3 = pd.read_csv('../data/preprocessing/QC-Demux/multiqc_data_2/multiqc_fastqc.txt', sep='\t')
df_demux = pd.concat([df1, df2, df3]).reset_index(drop=True)
for num in range(len(df_demux)):
    rename = df_demux.loc[num, 'Sample'].split('_')
    newname = rename[0]+'_'+rename[1]+'.'+rename[3]
    df_demux.loc[num, 'Sample'] = newname

# Hom many reads per sample after reorientation?
df1 = pd.read_csv('../data/preprocessing/QC-Reorient/multiqc_data/multiqc_fastqc.txt', sep='\t')
df2 = pd.read_csv('../data/preprocessing/QC-Reorient/multiqc_data_1/multiqc_fastqc.txt', sep='\t')
df3 = pd.read_csv('../data/preprocessing/QC-Reorient/multiqc_data_2/multiqc_fastqc.txt', sep='\t')
df_reorient = pd.concat([df1, df2, df3]).reset_index()

# How many reads per sample after cleaning?
df1 = pd.read_csv('../data/preprocessing/QC-Clean/multiqc_data/multiqc_fastqc.txt', sep='\t')
df2 = pd.read_csv('../data/preprocessing/QC-Clean/multiqc_data_1/multiqc_fastqc.txt', sep='\t')
df3 = pd.read_csv('../data/preprocessing/QC-Clean/multiqc_data_2/multiqc_fastqc.txt', sep='\t')
df_clean = pd.concat([df1, df2, df3]).reset_index()

In [3]:
df_raw.loc[:, ['Sample', 'pairs_processed', 'r1_with_adapters']]

,Sample,pairs_processed,r1_with_adapters
0,Psoil-1_S2_L001_R2_001,6318906,4811299
1,Psoil-2_S2_L001_R2_001,8795488,6700620
2,Psoil-3_S3_L001_R2_001,6869491,5121958


In [4]:
df_demux.loc[:, ['Sample', 'Total Sequences', 'avg_sequence_length', '%GC']]

,Sample,Total Sequences,avg_sequence_length,%GC
0,S1_rep1.R1,58356.0,257.138323,56.0
1,S1_rep1.R2,58356.0,257.455000,56.0
2,S1_rep2.R1,184683.0,253.747730,56.0
3,S1_rep2.R2,184683.0,254.219241,56.0
4,S1_rep3.R1,118356.0,263.372486,56.0
...,...,...,...,...
161,Swo_rep2.R2,39756.0,220.245598,57.0
162,Swo_rep3.R1,25460.0,232.204635,57.0
163,Swo_rep3.R2,25460.0,230.032600,57.0
164,unknown_L001.001,1747533.0,242.985675,55.0


In [5]:
df_reorient.loc[:, ['Sample', 'Total Sequences', 'avg_sequence_length', '%GC']]

,Sample,Total Sequences,avg_sequence_length,%GC
0,S1_rep1.R1,35104.0,274.385825,56.0
1,S1_rep1.R2,35104.0,293.861554,55.0
2,S1_rep2.R1,115009.0,273.958142,56.0
3,S1_rep2.R2,115009.0,293.901660,56.0
4,S1_rep3.R1,73253.0,274.087505,56.0
...,...,...,...,...
165,Swo_rep2.R2,515.0,283.728155,55.0
166,Swo_rep3.R1,44.0,241.931818,54.0
167,Swo_rep3.R2,44.0,269.272727,54.0
168,unknown.R1,291155.0,270.377153,54.0


In [6]:
df_clean.loc[:, ['Sample', 'Total Sequences', 'avg_sequence_length', '%GC']]

,Sample,Total Sequences,avg_sequence_length,%GC
0,S1_rep1.R1,35104.0,274.385825,56.0
1,S1_rep1.R2,35104.0,273.961543,56.0
2,S1_rep2.R1,115009.0,273.958142,56.0
3,S1_rep2.R2,115009.0,273.910442,56.0
4,S1_rep3.R1,73253.0,274.087505,56.0
...,...,...,...,...
165,Swo_rep2.R2,515.0,266.194175,55.0
166,Swo_rep3.R1,44.0,241.931818,54.0
167,Swo_rep3.R2,44.0,258.363636,54.0
168,unknown.R1,291155.0,270.377153,54.0


In [7]:
df = df_reorient.merge(df_demux, left_on='Sample', right_on='Sample')
df = df.merge(df_clean, left_on='Sample', right_on='Sample')
df = df.loc[[num for num in range(len(df)) if (num % 2) == 0], ['Sample', 'Total Sequences', 'Total Sequences_x', 'Total Sequences_y']]
df = df.rename(columns={'Total Sequences_y':'Demultiplexing', 
                   'Total Sequences_x':'Reorientation', 
                   'Total Sequences':'Cleaning'})
df.set_index('Sample')
df_melt = pd.melt(df, id_vars="Sample", var_name="Steps", value_name="Reads")
df.to_csv('../tables/QC_error0.1.tsv')
df_melt

,Sample,Steps,Reads
0,S1_rep1.R1,Cleaning,35104.0
1,S1_rep2.R1,Cleaning,115009.0
2,S1_rep3.R1,Cleaning,73253.0
3,S1_rep4.R1,Cleaning,103918.0
4,S1_rep5.R1,Cleaning,133619.0
...,...,...,...
235,Swc_rep2.R1,Demultiplexing,155005.0
236,Swc_rep3.R1,Demultiplexing,112654.0
237,Swo_rep1.R1,Demultiplexing,84397.0
238,Swo_rep2.R1,Demultiplexing,39756.0


In [10]:
#sns.set_style("whitegrid")
#sns.set_context("paper")
sns.catplot(y='Sample', x='Reads', hue='Steps', data=df_melt, kind='bar', orient='h')
plt.show()

In [9]:
df

,Sample,Cleaning,Reorientation,Demultiplexing
0,S1_rep1.R1,35104.0,35104.0,58356.0
2,S1_rep2.R1,115009.0,115009.0,184683.0
4,S1_rep3.R1,73253.0,73253.0,118356.0
6,S1_rep4.R1,103918.0,103918.0,208602.0
8,S1_rep5.R1,133619.0,133619.0,243235.0
...,...,...,...,...
150,Swc_rep2.R1,4583.0,4583.0,155005.0
152,Swc_rep3.R1,42543.0,42543.0,112654.0
154,Swo_rep1.R1,48328.0,48328.0,84397.0
156,Swo_rep2.R1,515.0,515.0,39756.0
